In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java.
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables.
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession.
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-01 09:32:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  2.18MB/s    in 0.4s    

2022-03-01 09:32:01 (2.18 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame.
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

instrument_df = spark.read.option("header", "true").option("sep", "\t").csv("file://" + SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"))
instrument_df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [5]:
# Count the number of records (rows) in the dataset.
instrument_df.count()

904765

In [6]:
# Drop all nulls.
instrument_df.dropna()

DataFrame[marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [7]:
# Find the total amount of customers who reviewed.

cust_groupby = instrument_df.groupBy("customer_id").count()
cust_groupby.orderBy("customer_id").select("customer_id", "count").show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   10000081|    1|
|   10000121|    2|
|   10000128|    1|
|   10000162|    1|
|   10000206|    1|
|   10000304|    1|
|   10000313|    1|
|   10000327|    2|
|    1000037|    2|
|   10000388|    1|
|    1000039|    1|
|   10000504|    1|
|   10000554|    1|
|   10000589|    1|
|   10000631|    1|
|   10000647|    1|
|   10000655|    1|
|   10000675|    1|
|   10000735|    1|
|   10000800|    8|
+-----------+-----+
only showing top 20 rows



In [8]:
cust_groupby.select(["customer_id"]).describe().show()

+-------+-------------------+
|summary|        customer_id|
+-------+-------------------+
|  count|             573149|
|   mean|2.683409559563569E7|
| stddev|1.559017532437193E7|
|    min|           10000081|
|    max|            9999947|
+-------+-------------------+



In [21]:
#Add the column customer_count to instrument_df.

from pyspark.sql.functions import lit

instrument_df = instrument_df.withColumn('customer_count', lit(573149))
instrument_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|customer_count|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|        573149|
|         US|   14640079| RZSL0BALIYUNU|B003LRN5

In [10]:
# Create user dataframe to match active_user table.
review_id_table2 = instrument_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"]).distinct()
review_id_table2.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3SQLJ4PFRBJ7A|     163404|B0044G7D8I|     734974703| 2015-08-31|
|R2I62GX0HLI3PW|    1611899|B00DYPDBY0|      95914319| 2015-08-31|
| R7DTS8AA3UOQW|   15439392|B00D6RMFG6|     406305804| 2015-08-31|
|R2M643SEPPUB29|   50550105|B009NN39YY|     533160261| 2015-08-31|
|R13XD9APWPDDCV|   41146239|B003GEBGA0|     934931360| 2015-08-31|
|R2FKNKI9UINYDQ|   13761624|B007J49GPK|     575084461| 2015-08-31|
|R1JRDWL6T0FTW4|   46308489|B00432PKDS|      83395415| 2015-08-31|
|R1L05FBIZU70HQ|   29341703|B00VHKMK64|      57787460| 2015-08-30|
|R2ESKIJVZFMMOZ|    7566541|B003B080LW|     459750077| 2015-08-30|
|R249JMOJMB23FQ|   18850955|B00CMKFO66|     417659397| 2015-08-30|
|R166DR1U8B3DAC|   41410353|B0009EU0S4|      36879237| 2015-08-30|
|R3ARA94XDNTZ6L|    4089808|B00SM4KA2E|     882569609| 2015-08

In [13]:
# Create user dataframe to match products table.
products2 = instrument_df.select(["product_id", "product_title"]).distinct()
products2 = products2.dropDuplicates(["product_id"]) 
products2.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0577088726|Sisyphus Redux fo...|
|0594478944|Nook Audio OE250 ...|
|0634010263|Hal Leonard Maná ...|
|0634029355|Hal Leonard Eric ...|
|0634056727|Hal Leonard My Fi...|
|0692218645|Ukulele Chord Cha...|
|0739050583|Alfred Billy Ward...|
|073906407X|Alfred Alfred's P...|
|0739075934|Alfred BILLY MART...|
|0739081381|Brandi Carlile Th...|
|0739086731|Harry Potter -- S...|
|0757918573|WB Wooten & Baile...|
|075799380X|WB Acoustic Blues...|
|0786615303|Premium Clarke Ti...|
|0786647272|The Best of Acous...|
|0786647434|The Sweetone Tinw...|
|0793525292|Hal Leonard The B...|
|0793536227|Hal Leonard Eric ...|
|0793543169|Hal Leonard Best ...|
|0793587395|Hal Leonard Third...|
+----------+--------------------+
only showing top 20 rows



In [14]:
# Create user dataframe to match customers table.
customers2 = instrument_df.select(["customer_id", "customer_count"]).distinct()
customers2.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11692003|        573149|
|   14536626|        573149|
|   51049664|        573149|
|   20160691|        573149|
|   14305160|        573149|
|   19528917|        573149|
|    3207461|        573149|
|   34550088|        573149|
|    4361068|        573149|
|    8558726|        573149|
|   28646597|        573149|
|   47377123|        573149|
|    4754795|        573149|
|   29694582|        573149|
|   22681561|        573149|
|   23295386|        573149|
|    4052476|        573149|
|   39710984|        573149|
|   51136140|        573149|
|   14408152|        573149|
+-----------+--------------+
only showing top 20 rows



In [15]:
# Create user dataframe to match vine table.
vine_table2 = instrument_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]).distinct()
vine_table2.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R31OIOD3CP2FB1|          4|            0|          0|   N|
|R305KWI3KUXZXL|          1|            0|          1|   N|
| RP2ANS7W0GPWY|          5|            0|          0|   N|
|R39BU4Q0BMH4X5|          1|            1|          2|   N|
|R1RHM0E24VMWTU|          5|            0|          0|   N|
| RUE8HYKC651I9|          5|            0|          1|   N|
|R1ZFEOXT6R71AD|          3|            0|          0|   N|
|R16HC2LDVNEZG5|          4|            0|          1|   N|
|R32TGGBBP6XLXI|          4|            0|          0|   N|
|R1VWTMMQJLCYS5|          5|            0|          0|   N|
| RBTTLEGPEIX19|          5|            0|          0|   N|
|R1IQ99II3ARA3H|          5|            0|          2|   N|
|R3VWE1XJGT4YVW|          4|            0|          0|   N|
|R10USIE781BXF1|          3|            

In [16]:
# Configuration for RDS instance.
mode="append"
jdbc_url = "jdbc:postgresql://XXXXXXXX:5432/big_data_challenge_RDS_1"
config = {"user":"XXXXXXXX",
          "password": "XXXXXXXX",
          "driver":"org.postgresql.Driver"}

In [17]:
# Write review_id_table DataFrame to table.

review_id_table2.write.jdbc(url=jdbc_url, table='review_id_table2', mode=mode, properties=config)

In [18]:
# Write products DataFrame to table.

products2.write.jdbc(url=jdbc_url, table='products2', mode=mode, properties=config)

In [19]:
# Write products DataFrame to table.

customers2.write.jdbc(url=jdbc_url, table='customers2', mode=mode, properties=config)

In [20]:
# Write vine DataFrame to table.

vine_table2.write.jdbc(url=jdbc_url, table='vine_table2', mode=mode, properties=config)